In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import glob
import cv2
from torchvision.transforms import ToTensor, Resize
import random
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Veri Kümesinin indirilmesi
#!git clone https://github.com/ardamavi/Sign-Language-Digits-Dataset.git

In [ ]:
class MyDataset(Dataset):
    def __init__(self, train):
        # Veri kümesinin yolu
        data_path = "dataset"

        # Veri kümesine ait klasördeki her bir JPG uzantılı resmin dosya yolu bir listeye kaydedildi.
        self.data_list = glob.glob(data_path + "/*/*.JPG")

        # Liste karıştırıldı.
        random.shuffle(self.data_list)

        # train True ise eğitim
        # train False ise doğrulama verim kümesi olacak şekilde ayarlandı.
        if train == True:
            self.data_list = self.data_list[:1600]
        else:
            self.data_list = self.data_list[1600:]

    def __getitem__(self, index):
        # görüntü dosyadan okunur.
        image = cv2.imread(self.data_list[index])

        # görüntü pytorch tensor veri tipine dönüştürülür ve 64x64 boyutuna indirgenir
        image = ToTensor()(image)
        image = Resize((64,64))(image)

        # dosya yolu bilgisinden etiket değeri elde edilir.
        label = self.data_list[index].split('\\')[1]
        return image, int(label)
  
    def __len__(self):
        return len(self.data_list)

In [ ]:
ds_train = MyDataset(train = True)
ds_val = MyDataset(train = False)

In [ ]:
dl_train = DataLoader(ds_train,batch_size=32,shuffle=True)
dl_val = DataLoader(ds_val,batch_size=32,shuffle=True)

In [ ]:
# Konvolüsyonel (evrişimli) bir sinir ağı tasarlandı.
# Bir katmanın çıktı sayısı diğer katman için girdi sayısı olacak.
# Resmimiz 3 kanaldan oluştuğu için ilk conv_layer'ın in_channels parametresi 3 olarak ayarlandı.

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3,padding=1)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3,padding=1)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3,padding=1)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        # Her pooling katmanında resim yarı boyutuna düşer. 2 pooling katmanı kullandık.
        # 64x64'lık resim -> 16x16'ya dönüşür.
        # full connected layer'ın girdi sayısı : 16x16x64 = 16384 olarak hesaplanır. Buradaki 64 son conv_layer'ın çıktı sayısıdır.
        self.fc1 = nn.Linear(16384, 1024)

        # Tahmin edilen sınıf sayısı 10 olduğu için modelin çıktı sayısı 10 olarak ayarlandı.
        self.fc2 = nn.Linear(1024, 10)
    
    def forward(self, x):
        # x verisi ilk katmanına girdi olarak verildikten sonra,
        # her bir katmanın çıktısı bir sonraki katmanın girdisi olarak ayarlandı.
        out = self.conv_layer1(x)
        out = F.relu(out)
        out = self.conv_layer2(out)
        out = F.relu(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = F.relu(out)
        out = self.conv_layer4(out)
        out = F.relu(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
model = MyModel()

In [ ]:
# model cpu'dan gpu'ya aktarılır.

#model = model.to('cuda')

In [ ]:
# kayıp fonksiyonu
loss_fn = nn.CrossEntropyLoss()

# Optimize edici olarak stochastic gradient descent seçtik, parametre olarak 
# model parametrelerini ve öğrenme oranını alır
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Modeli test etmek için eğitimde kullanılmayan verilerle oluşturduğumuz
# dl_val kullanılarak modelin testi gerçekleştirildi. 
# dogrulama() fonksiyonu çağrıldığında modelin test doğruluğunu verecektir.

def dogrulama():
    correct = 0
    with torch.no_grad():  # İleri yayılımda gradyanların hesaplanmaması için kullanılır. Gradyan hesabı eğitim için gerekli. gradyan hesabının yapılmaması için test aşamasında bu yapıyı kullanıyoruz.
        for data, label in dl_val:
            tahmin = model(data)   # görüntüler modelden geçirilir ve tahmin değerleri elde edilir.
            correct += (tahmin.argmax(1) == label).type(torch.float).sum().item()  # kaç tahmin doğru olarak bulundu

    sonuc = correct / len(ds_val)  # doğru olarak bulunan tahminlerin toplam veri sayısına oranı
    return sonuc

In [ ]:
epoch = 50

for idx in range(epoch):
    for data, label in dl_train:

        # her iterasyonda batch_size miktarınca görüntü modelden geçirilerek her 
        # görüntü için tahmin değerleri hesaplanıyor.
        tahmin = model(data)

        # elde edilen tahmin değerlerinin olması gereken değerlerden ne kadar uzak 
        # olduğuna dair kayıp hesaplanıyor.
        loss = loss_fn(tahmin, label)

        # Backpropagation (geri yayılım ve ağırlıkların güncellenmesi )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Her epoch'ta kullanıcı bilgilendiriliyor.
    print("Epoch :",idx, "Loss :", loss.item(), "val acc", dogrulama())

In [ ]:
# Eğittiğimiz modelin ağırlıklarını diske kaydediyoruz.

torch.save(model.state_dict(), 'model_weights.pth')